# LGP Defect detection in AI Core -  Part 1

The figure below summarizes the different steps one needs to go through to train and deploy a ML model in AI Core.

<img src="../../../../resources/AICoreMLOps.png" width="900">

For this training, we have already took care of most of the steps in Section 0 - One-time AI Core configuration. We have already provisioned AI Core and AI Launchpad for you, and we have already linked a Github repository to the AI Core tenant.  

In this notebook we will then assumed that Step 0.0 to Step 0.4 have been already done and we will guide you through:

* **Connect a Docker registry to an AI Core instance** using the AI API SDK (step 0.5 in the figure)
* **Upload data on S3 and create a resource group** (step 1 in the figure)

<span style="color:red">For your reference, a complete example covering all the steps of the one-time AI Core configuration is available in </span>[this notebook](../notebooks/defect_detection_aicore_notebook_one_time_config.ipynb) .


### Before getting started: color conventions

The comments within the notebook will guide you to the required steps. Pay attention to the color conventions:

* <span style="color:magenta"> **Magenta text**  </span> indicates that you have to open certain json files and modify them according to your own set up, for instance you can be asked to enter credentials for a certain system, change names for the variables etc.  
* <span style="color:blue"> **Blue text**  </span> indicates that you have to execute commands on a terminal. 
* <span style="color:green"> **Green text** </span> indicates that you are asked to modify something in the following notebook cell. 


# Create an AI API client instance

In [ ]:
from ai_api_client_sdk import ai_api_v2_client
from ai_api_client_sdk.ai_api_v2_client import AIAPIV2Client
import json

In [ ]:
from ai_api_client_sdk import ai_api_v2_client

First of all, we need to create an AI API client instance, which will allow us to interact with our SAP AI Core tenant. <span style="color:magenta">Before executing the code, edit **[aic_service_key.json](./files/aic_service_key.json)** and enter the right credentials of your AI Core tenant</span>.

In [ ]:
# Your service key JSON file relative to your aicore instance
aic_service_key_path = './files/aic_service_key.json'

# Loads the service key file
with open(aic_service_key_path) as ask:
    aic_service_key = json.load(ask)

# Creating an AI API client instance
ai_api_client = AIAPIV2Client(
    base_url = aic_service_key["serviceurls"]["AI_API_URL"] + "/v2", # The present AI API version is 2
    auth_url=  aic_service_key["url"] + "/oauth/token",
    client_id = aic_service_key['clientid'],
    client_secret = aic_service_key['clientsecret']
)

##  Connect your Docker Registry to SAP AI Core

Next, let's use the AI API client to connect your  we need to create an AI API client instance, which will allow us to interact with our SAP AI Core tenant. <span style="color:magenta">Before executing the code, edit </span> **[docker_secret.json](./files/docker_secret.json)** <span style="color:magenta">with your docker credentials</span>:
* <span style="color:magenta">name:</span> complete with your user number
* <span style="color:magenta">username:</span> your docker username
* <span style="color:magenta">password:</span> your docker access token
</span>.

In [ ]:
docker_secret_file_path = './files/docker_secret.json'

# Loads the json file
with open(docker_secret_file_path) as dsf:
    docker_secret = json.load(dsf)

response = ai_api_client.rest_client.post(
    path="/admin/dockerRegistrySecrets",
    body={
        "name": docker_secret["name"],
        "data": docker_secret["data"]
    }
)
print(response)

# Upload data on S3 and connect them to a Resource Group

### Create a new Resource Group

Resource Groups represent a virtual collection of related resources within the scope of one SAP AI Core tenant. Let's create a specific resource group for the defect detection exercise with the AI API client. The new resource group will then appear also on the AI Lauchpad UI. 


Let's first use the **get** api have a look at the resource groups already present in the tenant

In [ ]:
ai_api_client.rest_client.get(
    path=f"/admin/resourceGroups"
)

Let's assign a name for the new resource group <span style="color:green"> Modify the assignment below according to your participant ID, so that it won't confict with other participants </span>

In [ ]:
resource_group="defect-det-u0001"

Use the **post** API to create the resource group

In [ ]:
ai_api_client.rest_client.post(
    path="/admin/resourceGroups",
    body={
        "resourceGroupId": resource_group 
    }
)

We can now verify the new resource group is now present on the list

In [ ]:
ai_api_client.rest_client.get(
    path=f"/admin/resourceGroups/{resource_group}"
)

### Upload the data on AWS S3 and register it to the Resource Group

We are using AWS S3 Object Store as a cloud storage for our AI Core datasets and models. 

For this exercise, we have already loaded our input dataset on a S3 Storage bucket for you. If you had to do this on your own, you could always get AWS S3 Bucket from either of these two ways:
- through SAP BTP Cockpit;
- through AWS. Refer AWS User Guide to S3.

You would then need to install AWS CLI and upload your dataset through a command like the following:
```sh
aws s3 cp --recursive /local/path/to/data s3://your-bucket-id/image/data/
```

Once the data are in the AWS S3, we need to **register the Object Store to SAP AI Core resource group**. In order to to that we prepare a json file with the needed AWS S3 credentials and we use a proper AI API in order to register it. <span style="color:magenta"> Please enter your AWS S3 Object Store credentials in  </span>  **[s3_service_key.json](./files/s3_service_key.json)** before executing the code. Additionally <span style="color:green"> Modify the name of the S3 secret in the cell below bucket-uxxx according to your participant ID, so that it won't confict with other participants </span>


In [ ]:
# Loads your service key
s3_service_key_path = "./files/s3_service_key.json"

# Loads the service key file
with open(s3_service_key_path) as s3sk:
    s3_service_key = json.load(s3sk)

default_secret = {
    "name": "bucket-u0001", # To be modified
    "type": "S3",
    "endpoint": s3_service_key["host"],
    "bucket": s3_service_key["bucket"],
    "pathPrefix": "image",
    "region": s3_service_key["region"],
    "data": {
        "AWS_ACCESS_KEY_ID": s3_service_key["access_key_id"],
        "AWS_SECRET_ACCESS_KEY": s3_service_key["secret_access_key"]
    }
}

# Call the api
ai_api_client.rest_client.post(
    path="/admin/objectStoreSecrets",
    body = default_secret, # defined above
    resource_group = resource_group
)

### Check out [defect-detection-part2.ipynb](./defect-detection-part2.ipynb) to continue with the exercise.